In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('jobfair_train.csv')
dataset = dataset.sort_values(by='league_id')
dataset = dataset.drop('registration_country', axis=1)
dataset = dataset.drop('registration_platform_specific', axis=1)
dataset = dataset.drop('season', axis=1)

test_dataset = pd.read_csv('jobfair_test.csv')
test_dataset = test_dataset.sort_values(by='league_id')
test_dataset = test_dataset.drop('registration_country', axis=1)
test_dataset = test_dataset.drop('registration_platform_specific', axis=1)
test_dataset = test_dataset.drop('season', axis=1)

In [23]:
'''Dynamic_Payment_Segment'''
new_dynamic = {'0) NonPayer': 0, '1) ExPayer': 1, '2) Minnow': 2, '3) Dolphin': 3, '4) Whale': 4}
dataset['dynamic_payment_segment'] = dataset['dynamic_payment_segment'].map(new_dynamic)
test_dataset['dynamic_payment_segment'] = test_dataset['dynamic_payment_segment'].map(new_dynamic)

'''Global_Competition_Level'''
dataset['global_competition_level'].fillna(0, inplace = True)
test_dataset['global_competition_level'].fillna(0, inplace = True)

# Help

In [24]:
import copy
def one_post_process(y_p):
    y_p_copy = y_p[:]
    temp = np.argsort(np.array(y_p))
    for i in range(len(y_p)):
        y_p_copy[temp[i]] = i + 1
    return y_p_copy

def post_sorting(y_pred, length):
    y_prediction = copy.deepcopy(y_pred)
    n = int(len(y_prediction) / length)

    for i in range(n):
        y_prediction[i*length:(i+1)*length] = one_post_process(y_prediction[i*length:(i+1)*length])

    return y_prediction

# Prediction

In [25]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(dataset.iloc[:,:-1].values)
y_train = dataset.iloc[:, -1].values
X_test_scaled = sc.transform(test_dataset.iloc[:,:].values)

In [26]:
from xgboost import XGBRegressor
regressor = XGBRegressor(reg_lambda = 0.2, learning_rate = 0.1, gamma = 0,reg_alpha = 0,\
                          subsample = 0.9, max_depth = 7, min_child_weight = 50, n_estimators = 85)

In [ ]:
regressor.fit(X_train_scaled, y_train)

In [28]:
y_pred_test = regressor.predict(X_test_scaled)
y_pred_test_sorted = post_sorting(y_pred_test, 14)

# Saving a results

In [29]:
test_dataset['league_rank'] = y_pred_test_sorted

In [ ]:
#print(test_dataset.head(14))

In [31]:
new_dataset = test_dataset[['club_id', 'league_rank']]
new_dataset = new_dataset.sort_values(by='club_id')
new_dataset.to_csv("league_rank_predictions.csv", index = False)